In [1]:
import os
import re
import csv
import string
import unicodedata
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("data.csv",  encoding='utf8', low_memory=False )

In [3]:
data.shape

(4410, 12)

In [4]:
data.tail(2)

,_id,collective,date,file_id,judicature,participants,report_receipt_at,sentence_id,text,topic,url,vocabulary
4408,corteconstitucional_A. 175A/19,"{""entity"": {""$oid"": ""5fa5017c5821d011a2a92372""}}",2019-04-03T00:00:00.000Z,T-7071794,"[{""name"": ""ALBERTO ROJAS RÍOS"", ""rol"": ""PONENT...","[{""name"": ""DARIO GOMEZ SUAREZ"", ""role"": ""PLAIN...",2019-07-08,A. 175A/19,Auto 175A/19\n \n \nReferencia: Expediente T-7...,El accionante le solicitó a la Corporación dec...,https://www.corteconstitucional.gov.co/relator...,"[{""text"": ""auto"", ""qty"": 7}, {""text"": ""referen..."
4409,corteconstitucional_A. 121/19,"{""entity"": {""$oid"": ""5fa501875821d011a2a93763""}}",2019-03-13T00:00:00.000Z,T-6843600,"[{""name"": ""GLORIA STELLA ORTIZ DELGADO"", ""rol""...","[{""name"": ""COMISION INTERECLESIAL DE JUSTICIA ...",2019-03-21,A. 121/19,Auto\r\n121/19\n \n \nReferencia: expediente T...,Con el presente auto se suspenden los términos...,https://www.corteconstitucional.gov.co/relator...,"[{""text"": ""auto"", ""qty"": 17}, {""text"": ""refere..."


In [5]:
data = data.drop(["judicature", "participants", "collective", "_id", "topic", "url", "vocabulary", "date"], axis=1)

In [6]:
data.head()

,file_id,report_receipt_at,sentence_id,text
0,T-3140852,2012-08-08,T-017/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-017/12\...
1,T-3183107,2012-05-18,T-016/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia\r\nT-016/...
2,T-3187308,2013-05-07,T-014/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia\r\nT-014/...
3,T-3182540,2012-05-18,T-015/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-015/12\...
4,T-2719663,2011-05-10,T-012/11,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-012/11\...


In [7]:
data.head(1)

,file_id,report_receipt_at,sentence_id,text
0,T-3140852,2012-08-08,T-017/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-017/12\...


In [8]:
data["text"] = data["text"].str.lower()

In [9]:
len(data["text"])

4410

In [10]:
data

,file_id,report_receipt_at,sentence_id,text
0,T-3140852,2012-08-08,T-017/12,república de colombia\n\n\nsentencia t-017/12\...
1,T-3183107,2012-05-18,T-016/12,república de colombia\n\n\nsentencia\r\nt-016/...
2,T-3187308,2013-05-07,T-014/12,república de colombia\n\n\nsentencia\r\nt-014/...
3,T-3182540,2012-05-18,T-015/12,república de colombia\n\n\nsentencia t-015/12\...
4,T-2719663,2011-05-10,T-012/11,república de colombia\n\n\nsentencia t-012/11\...
...,...,...,...,...
4405,T-6403234,2019-04-08,A. 177/19,auto 177/19\n \nreferencia:\r\nexpediente t-64...
4406,T-6945381,2019-05-27,A. 177A/19,auto\r\n177a/19\n \n \nreferencia:\r\nexpedien...
4407,T-7064392 Y T-7073283,2019-04-08,SU.139/19,sentencia\r\nsu139/19 \n \naccion de tutela co...
4408,T-7071794,2019-07-08,A. 175A/19,auto 175a/19\n \n \nreferencia: expediente t-7...


In [11]:
new_data = pd.DataFrame([], columns = ['file_id', 'antecedentes' , 'decision'])

In [12]:
new_data

,file_id,antecedentes,decision


In [13]:
def quitar_tildes(cadena):
    s = ''.join((c for c in unicodedata.normalize('NFD',cadena) if unicodedata.category(c) != 'Mn'))
    return s

In [14]:
def get_segment(data):
    lista_df = []
    for index, row in data.iterrows():
        texto = re.split(
            r'[\n]+[(^i|^v|)]+\.\s{0,}[(\w)]{0,}?', 
            quitar_tildes(row["text"])
        )
        if len(texto) > 4:
            lista_df.append({"file_id": row["file_id"] , "antecedentes": texto[1], "decision": texto[4]})

    return lista_df

In [15]:
new_data = pd.DataFrame(get_segment(data), columns = ["file_id", 'antecedentes' , 'decision'])

In [16]:
new_data

,file_id,antecedentes,decision
0,T-3187308,antecedentes\n \n1. hechos\n \n1. el 27 de jul...,decision\n \ncon\r\nfundamento en las consider...
1,T-2719663,antecedentes\n \n1. hechos\n \nel defensor ...,"decision\n \nen merito de lo expuesto, la\r\ns..."
2,T-3176539,antecedentes\n \nel ocho (8) de junio de dos m...,"decision\n \nen merito de lo expuesto, la sala..."
3,T-3180007,"antecedentes\n \nel senor juan antonio, a trav...",consideraciones de la corte constitucional\n \...
4,T-2764991 Y 2822307,antecedentes\n \n1. hechos\n \nlos hechos rela...,"decision \n \nen merito de lo expuesto, la sal..."
...,...,...,...
1857,T-7128811,antecedentes\n \n1. hechos\n \n\n1.1 la senora...,que no esten\r\nexpresamente incluidos en el p...
1858,T-6960341,antecedentes \n \n1. hechos y relato contenido...,pruebas decretadas en sede de revision \n \nel...
1859,T-7147012,antecedentes\n \nherley\r\nramirez alzate pres...,existencia\r\nde un peligro especifico e indiv...
1860,T-7103649,antecedentes\n \na. hechos y pretensiones\n \n...,"decision\n \nen merito de lo expuesto, la sala..."


In [17]:
new_data.shape

(1862, 3)

In [18]:
new_data.to_csv('splid_data.csv', sep=",",quotechar='"', quoting=csv.QUOTE_ALL, encoding='utf-8')

In [19]:
new_data.to_csv(r'splid_data.txt', header=None, index=None, sep=' ', mode='a')

In [20]:
def quitar_espacios(text):
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\r', ' ', text)
    text = re.sub('\xa0', ' ', text)

    return text

new_data["decision"] = new_data.decision.apply(lambda x: quitar_espacios(x))
new_data["antecedentes"] = new_data.antecedentes.apply(lambda x: quitar_espacios(x))

In [21]:
def quita_signos(text):
    puntuacion = string.punctuation
    for item in puntuacion:
        text = text.replace(item, "")
    return text

#new_data["decision"] = pd.DataFrame(new_data.decision.apply(lambda x: quita_signos(x)))
#new_data["antecedentes"] = pd.DataFrame(new_data.antecedentes.apply(lambda x: quita_signos(x)))

In [22]:
new_data[1:2]["antecedentes"]

1    antecedentes   1.    hechos   el defensor del ...
Name: antecedentes, dtype: object

In [23]:
new_data["decision"][50][0:2000]

'decision   en merito de lo expuesto, la  sala novena de revision de la corte constitucional, administrando justicia en  nombre del pueblo y por mandato de la constitucion,     resuelve:   primero.- revocar la sentencia  proferida el 27 de septiembre  de 2011 por la sala de casacion laboral de la  corte suprema de justicia que a su turno habia confirmado la sentencia dictada  el 10 de agosto de 2011 por sala de casacion civil de la corte suprema de  justicia que habia negado el amparo constitucional solicitado. en su lugar, conceder  la tutela por violacion del derecho al debido proceso.   segundo.- dejar sin efecto  las actuaciones surtidas por el juzgado cuarto civil del circuito de santa  marta y por la sala civil-familia del tribunal superior de la misma ciudad en  el proceso ejecutivo hipotecario seguido por el ahorramas hoy banco av villas  s.a. contra la empresa construcciones dihago ltda. representada por alejandro  habeych gonzalez y contra las personas ana maria barrios queve

In [24]:
new_data["decision"][50][1:1000]

'ecision   en merito de lo expuesto, la  sala novena de revision de la corte constitucional, administrando justicia en  nombre del pueblo y por mandato de la constitucion,     resuelve:   primero.- revocar la sentencia  proferida el 27 de septiembre  de 2011 por la sala de casacion laboral de la  corte suprema de justicia que a su turno habia confirmado la sentencia dictada  el 10 de agosto de 2011 por sala de casacion civil de la corte suprema de  justicia que habia negado el amparo constitucional solicitado. en su lugar, conceder  la tutela por violacion del derecho al debido proceso.   segundo.- dejar sin efecto  las actuaciones surtidas por el juzgado cuarto civil del circuito de santa  marta y por la sala civil-familia del tribunal superior de la misma ciudad en  el proceso ejecutivo hipotecario seguido por el ahorramas hoy banco av villas  s.a. contra la empresa construcciones dihago ltda. representada por alejandro  habeych gonzalez y contra las personas ana maria barrios queved

In [25]:
#from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *
#import sparknlp
#import pyspark.sql.functions as F

#spark = sparknlp.start()
#spark

In [26]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [27]:
import sparknlp
spark = sparknlp.start()
from sparknlp.pretrained import PretrainedPipeline
spark


In [28]:
explain_document_pipeline = PretrainedPipeline("explain_document_md", lang = 'es')

explain_document_md download started this may take some time.
Approx size to download 453.9 MB
[OK!]


In [66]:
annotations = explain_document_pipeline.annotate(new_data["decision"][51])
list(annotations.keys())

['entities',
 'lemma',
 'document',
 'pos',
 'token',
 'ner',
 'embeddings',
 'sentence']

In [30]:
print('articulo' == annotations['token'])

False


In [31]:
def count_word(word):
    count=0
    for item in annotations["token"]:
        if word == item or f"{word}s" == item:
            count = count + 1
    return count

In [34]:
print("=====================")
print("Articulos:....... " + str(count_word('articulo')))
print("=====================")
print("Decretos:....... " + str(count_word('decreto')))
print("=====================")
print("Sentencias:....... " + str(count_word('sentencia')))
print("=====================")
print("Folios:....... " + str(count_word('folio')))
print("=====================")

Articulos:....... 12
Decretos:....... 5
Sentencias:....... 64
Folios:....... 6


In [82]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
sentencerDL = SentenceDetectorDLModel\
  .pretrained("sentence_detector_dl", "en") \
  .setInputCols(["document"]) \
  .setOutputCol("sentences")

sd_pipeline = PipelineModel(stages=[documenter, sentencerDL])
sd_model = LightPipeline(sd_pipeline)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [293]:
def processed_step_by_step(index):
    
    annotations = explain_document_pipeline.annotate(new_data["decision"][51])

    
    documenter = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

    sentencerDL = SentenceDetectorDLModel\
      .pretrained("sentence_detector_dl", "en") \
      .setInputCols(["document"]) \
      .setOutputCol("sentences")

    sd_pipeline = PipelineModel(stages=[documenter, sentencerDL])
    sd_model = LightPipeline(sd_pipeline)
    
    return sd_model
    

In [294]:
def get_list_sentenses_by_model(text_count):
    list_sentenses = []
    for sentense in sd_model.fullAnnotate(text_count)[0]["sentences"]:
        list_sentenses.append(sentense.result)
    return list_sentenses

In [295]:
def extraer_articulos(sentences, topic):
    lista_df = []
    for item in sentences:
        regx = f"({topic}|{topic}s)[ ]([\s\S]*?)\."
        expretion = re.compile(r"{}".format(regx))
        result = expretion.findall(item)
        for item_normalize in list(map(" ".join, result)):
            lista_df.append(item_normalize)
    return lista_df

In [296]:

def get_list_sentenses():
    list_sentenses = []
    for sentense in sd_model.fullAnnotate(text)[0]["sentences"]:
        list_sentenses.append(sentense.result)
    return list_sentenses

In [297]:
new_dataframe_to_graph = pd.DataFrame(columns = ["folios", "reference"])

In [298]:
new_data

,file_id,antecedentes,decision
0,T-3187308,antecedentes 1. hechos 1. el 27 de julio d...,decision con fundamento en las consideracio...
1,T-2719663,antecedentes 1. hechos el defensor del ...,"decision en merito de lo expuesto, la sala ..."
2,T-3176539,antecedentes el ocho (8) de junio de dos mil...,"decision en merito de lo expuesto, la sala t..."
3,T-3180007,"antecedentes el senor juan antonio, a traves...",consideraciones de la corte constitucional 1...
4,T-2764991 Y 2822307,antecedentes 1. hechos los hechos relatado...,"decision en merito de lo expuesto, la sala ..."
...,...,...,...
1857,T-7128811,antecedentes 1. hechos 1.1 la senora kari...,que no esten expresamente incluidos en el pbs...
1858,T-6960341,antecedentes 1. hechos y relato contenidos ...,pruebas decretadas en sede de revision el ...
1859,T-7147012,antecedentes herley ramirez alzate presento...,existencia de un peligro especifico e individ...
1860,T-7103649,antecedentes a. hechos y pretensiones 1. ...,"decision en merito de lo expuesto, la sala s..."


# Seccion DECISION [sentencias, articulos, folios, decretos]

In [313]:
text = new_data["decision"][28]

In [338]:
decision_new_data_folio = pd.DataFrame(columns = ["folios", "reference"])
decision_new_data_articulo = pd.DataFrame(columns = ['articulos', "reference"])
decision_new_data_decreto = pd.DataFrame(columns = ['decretos', "reference"])
decision_new_data_sentencia = pd.DataFrame(columns = ['sentencias', "reference"])

In [358]:
decision_new_data_folio['folios'] = extraer_articulos(get_list_sentenses()[27:128], "folio")

In [359]:
decision_new_data_decreto['decretos'] = extraer_articulos(get_list_sentenses(), "decreto")

In [360]:
decision_new_data_articulo['articulos'] = extraer_articulos(get_list_sentenses(), "articulo")

In [361]:
decision_new_data_sentencia['sentencias'] = extraer_articulos(get_list_sentenses()[27:128], "sentencia")

In [363]:
decision_new_data_articulo.tail(30)

,articulos
0,"articulos 86 y 241, numeral 9°, de la constit..."
1,"articulo 29 superior, dispone que el debido p..."
2,"articulo 209 superior), toda vez que a traves ..."
3,"articulo 86 superior, la accion de tutela es u..."
4,"articulo 6° del decreto 2591 de 1991, el cual..."
5,articulo 49 del codigo contencioso administrat...
6,articulo 58 y el decreto distrital 531 de 23 ...
7,articulo 58 y el decreto distrital 531 de 23 d...
8,articulo 28 del c
9,"articulo 17 de la ley 1333 de 2009, la privo d..."


In [357]:
folios = extraer_articulos(get_list_sentenses_by_model(new_data["decision"][28]), "folio")
folios

[]

In [352]:
decision_new_data_folio = pd.DataFrame(columns = ["folios"])
decision_new_data_articulo = pd.DataFrame(columns = ['articulos'])
decision_new_data_decreto = pd.DataFrame(columns = ['decretos'])
decision_new_data_sentencia = pd.DataFrame(columns = ['sentencias'])

def get_new_data(sentence):
    folios = extraer_articulos(get_list_sentenses_by_model(sentence), "folio")
    articulos = extraer_articulos(get_list_sentenses_by_model(sentence), "articulo")
    sentencias = extraer_articulos(get_list_sentenses_by_model(sentence), "sentencia")
    decretos = extraer_articulos(get_list_sentenses_by_model(sentence), "decreto")
    return folios, articulos, sentencias, decretos
    

In [384]:
dec = pd.DataFrame(columns = ["folios", "articulos", "sentencias", "decretos", "reference"])
for item in range(1, 1800):
    reference = new_data["file_id"][item]
    folios, articulos, sentencias, decretos = get_new_data(new_data["decision"][item])
    dec = dec.append(
        {
         "folios": articulos, 
         "articulos": articulos, 
         "sentencias": sentencias, 
         "decretos": decretos, 
         "reference": reference
        }, 
        ignore_index=True
    )
dec

,folios,articulos,sentencias,decretos,reference
0,"[articulo 36 del decreto 2591 de 1991, articul...","[articulo 36 del decreto 2591 de 1991, articul...",[sentencia y del expediente a la superintenden...,[decreto 2591 de 1991],T-2719663
1,"[articulo 36 del decreto 2591 de 1991, para l...","[articulo 36 del decreto 2591 de 1991, para l...",[sentencia proferida por el juzgado primero pe...,"[decreto 2591 de 1991, para los efectos alli ...",T-3176539
2,[articulos 86 y 241-9 de la constitucion poli...,[articulos 86 y 241-9 de la constitucion poli...,"[sentencia c-134 de 1994, en la que se declar...","[decreto 2591 de 1991, decreto 2591 de 1991, ...",T-3180007
3,[articulo 239 del codigo sustantivo del trabaj...,[articulo 239 del codigo sustantivo del trabaj...,[sentencia proferida el siete (7) de mayo de 2...,"[decreto 2591 de 1991, decreto 3135 de 1968, e...",T-2764991 Y 2822307
4,[articulos 86 y 241 numeral 9° de la constituc...,[articulos 86 y 241 numeral 9° de la constituc...,[sentencia proferida dentro del proceso de la ...,[decreto 2591 de 1991],T-3201980
...,...,...,...,...,...
1794,"[articulo 36 del decreto ley 2591 de 1991, art...","[articulo 36 del decreto ley 2591 de 1991, art...",[sentencias proferidas el tres (3) de septiemb...,"[decreto ley 2591 de 1991, decreto 2591 de 199...",T-7136911
1795,[articulo 55 (sic) del reglamento interno de ...,[articulo 55 (sic) del reglamento interno de ...,[sentencia t-284 de 2019[153] (expediente t-7...,[],T-7055614 Y OTRO ACUMULADOS
1796,[articulo 488 del codigo sustantivo del traba...,[articulo 488 del codigo sustantivo del traba...,"[sentencias t-426 de 2001 m, sentencias t-051 ...","[decreto 2591 de 1991, decreto 1730 de 2001: i...",T-7222037 Y OTRO ACUMULADOS
1797,"[articulos 86 y 241, numeral 9º, de la consti...","[articulos 86 y 241, numeral 9º, de la consti...",[sentencias proferidas dentro del proceso de l...,"[decreto 2591 de 1991, decreto 2591 de 1991 s...",T-6899349


In [377]:
dec["folios"][0][1]

'articulo 29 superior, dispone que el debido  proceso se aplicara a toda clase de actuaciones judiciales y administrativas  precisando, asi mismo, que nadie podra ser juzgado sino conforme a leyes  preexistentes al acto que se le imputa, ante juez o tribunal competente y con  observancia de la plenitud de las formas propias de cada juicio'

In [244]:
decision_new_data_articulo.head()

,articulos,reference


In [385]:
dec.shape

(1799, 5)

In [387]:
dec[300:310]

,folios,articulos,sentencias,decretos,reference
300,"[articulo 36 del decreto 2591 de 1991, articul...","[articulo 36 del decreto 2591 de 1991, articul...",[sentencia dictada en diciembre 19 de 2011 po...,[decreto 2591 de 1991],T-3353576
301,"[articulo 36 del decreto 2591 de 1991, articu...","[articulo 36 del decreto 2591 de 1991, articu...",[sentencia proferida el 22 de noviembre de 201...,"[decreto 2591 de 1991, decreto 2591 de 1991: ...",T-3363570
302,[],[],[],[],T-2919170
303,"[articulos 86 y 241, numeral 9º de la constitu...","[articulos 86 y 241, numeral 9º de la constitu...",[sentencia proferida dentro del proceso de la...,[decreto 2591 de 1991],T-3027375
304,"[articulos 86 y 241, numeral 9º, de la constit...","[articulos 86 y 241, numeral 9º, de la constit...",[sentencia t-786 del 14 de agosto de 2008 rei...,"[decreto 2591 de 1991, decreto 2274 de 1998- s...",T-2891721 Y OTRO ACUMULADO
305,"[articulo 241 de la norma fundamental, articu...","[articulo 241 de la norma fundamental, articu...",[sentencia anterior deben ser semejantes o pla...,[],T-2864851
306,"[articulo 228 de la constitucion, a lineamient...","[articulo 228 de la constitucion, a lineamient...","[sentencia t-304 de 2006, en la que se estable...","[decreto 190 de 12004 ""por medio del cual se c...",T-2991694
307,[],[],[],[],T-2972157
308,[articulos 86 y 241 de la constitucion politic...,[articulos 86 y 241 de la constitucion politic...,"[sentencia t-025 de 2004,[6] que existe una ob...",[decreto 2591 de 1991 y las demas disposicione...,T-2549402
309,[articulos 86 y 241 de la constitucion politic...,[articulos 86 y 241 de la constitucion politic...,[sentencia t-888 de 1999: [] se garantiza...,[decreto 2591 de 1991 y las demas disposicion...,T-2674656


In [391]:
dec.to_parquet('processed_data.gzip', compression='gzip')

In [51]:
decision_new_data_decreto.head()

,decretos
0,decreto ley 663 de 1993 (estatuto organico de...
1,decreto ley 663 de 1993 (estatuto organico de...
2,"decreto 856 de mayo 19 de 1999, (diario ofici..."
3,"decreto 2599 de 1998, especificando la denomi..."


In [52]:
decision_new_data_folio.head()

,folios


In [393]:
parquet_data = pd.read_parquet('processed_data.gzip')

In [394]:
parquet_data

,folios,articulos,sentencias,decretos,reference
0,"[articulo 36 del decreto 2591 de 1991, articul...","[articulo 36 del decreto 2591 de 1991, articul...",[sentencia y del expediente a la superintenden...,[decreto 2591 de 1991],T-2719663
1,"[articulo 36 del decreto 2591 de 1991, para l...","[articulo 36 del decreto 2591 de 1991, para l...",[sentencia proferida por el juzgado primero pe...,"[decreto 2591 de 1991, para los efectos alli ...",T-3176539
2,[articulos 86 y 241-9 de la constitucion poli...,[articulos 86 y 241-9 de la constitucion poli...,"[sentencia c-134 de 1994, en la que se declar...","[decreto 2591 de 1991, decreto 2591 de 1991, ...",T-3180007
3,[articulo 239 del codigo sustantivo del trabaj...,[articulo 239 del codigo sustantivo del trabaj...,[sentencia proferida el siete (7) de mayo de 2...,"[decreto 2591 de 1991, decreto 3135 de 1968, e...",T-2764991 Y 2822307
4,[articulos 86 y 241 numeral 9° de la constituc...,[articulos 86 y 241 numeral 9° de la constituc...,[sentencia proferida dentro del proceso de la ...,[decreto 2591 de 1991],T-3201980
...,...,...,...,...,...
1794,"[articulo 36 del decreto ley 2591 de 1991, art...","[articulo 36 del decreto ley 2591 de 1991, art...",[sentencias proferidas el tres (3) de septiemb...,"[decreto ley 2591 de 1991, decreto 2591 de 199...",T-7136911
1795,[articulo 55 (sic) del reglamento interno de ...,[articulo 55 (sic) del reglamento interno de ...,[sentencia t-284 de 2019[153] (expediente t-7...,[],T-7055614 Y OTRO ACUMULADOS
1796,[articulo 488 del codigo sustantivo del traba...,[articulo 488 del codigo sustantivo del traba...,"[sentencias t-426 de 2001 m, sentencias t-051 ...","[decreto 2591 de 1991, decreto 1730 de 2001: i...",T-7222037 Y OTRO ACUMULADOS
1797,"[articulos 86 y 241, numeral 9º, de la consti...","[articulos 86 y 241, numeral 9º, de la consti...",[sentencias proferidas dentro del proceso de l...,"[decreto 2591 de 1991, decreto 2591 de 1991 s...",T-6899349


# Propuesta de generar NER para documentos Legales en Colombia

In [162]:
tokens=["articulo", "16", "de", "la", "ley", "31", "de", "1992"]
pos_labels=['ART', 'NUM', 'NN', 'NN', 'NN', 'NUM', 'NN', 'YEAR', '.']
entity_labels=['ARTICULO','NUM-ARTICULO','STOP','LEY','NUM-LEY','STOP','LEY-ANNO','O','O']

In [163]:
labels_lines=''
for token,pos,label in zip(tokens,pos_labels,entity_labels):
    labels_lines+="{} {} {} {}\n".format(token, pos, pos, label)
print(labels_lines)

articulo ART ART ARTICULO
16 NUM NUM NUM-ARTICULO
de NN NN STOP
la NN NN LEY
ley NN NN NUM-LEY
31 NUM NUM STOP
de NN NN LEY-ANNO
1992 YEAR YEAR O



In [223]:
addrtext = '[15] el articulo 86 de la c. p. reza lo siguiente: toda persona tendra accion de Articulo 12,23, 23,23. tutela para reclamar ante los jueces,'    
newtext = addrtext  
ippat = re.compile(r'(articulo|articulos)[ ]([\s\S]*?)\.', flags=re.I | re.X)  
mtch = ippat.search(newtext) 
print(ippat.findall(newtext))
print(mtch)



[('articulo', '86 de la c'), ('Articulo', '12,23, 23,23')]
<re.Match object; span=(8, 28), match='articulo 86 de la c.'>


In [224]:
import re
line = "[15] el articulo 86 de la c. p. reza lo siguiente: toda persona tendra accion de Articulo 12,23, 23,23. tutela para reclamar ante los jueces,"
matches = re.findall('(articulo|articulos)[ ]([\s\S]*?)\.', line)
print(matches)

[('articulo', '86 de la c')]


In [255]:
import re
s = "[15] el articulo 86 de la c. p. reza lo siguiente: toda persona tendra accion de articulo 12,23, 23,23. tutela para reclamar ante los jueces, del folio, del decreto,."
r = re.compile(r'(articulo|articulos)[ ]([\s\S]*?)\.')
print(r.findall(s))

[('articulo', '86 de la c'), ('articulo', '12,23, 23,23')]


## Generador data

In [256]:
validate =  [int, str, "articulo", "folio", "sentencia", "decreto"]


In [294]:
validate_label = {
    "articulo": "B-ART",
    "folio": "FOL",
    "sentencia": "SEN",
    "decreto": "DEC",
    "ley": "LEY",
    "num": "NUM",
    False: "MIS"
}
validate_label

{'articulo': 'B-ART',
 'folio': 'FOL',
 'sentencia': 'SEN',
 'decreto': 'DEC',
 'ley': 'LEY',
 'num': 'NUM',
 False: 'MIS'}

In [295]:
validate_label_mix = {
    "sw": "SW", # refactorizar, nltk StopWords
}
validate_label_mix

{'sw': 'SW'}

In [296]:
def clean_word(word):
    _ = quita_signos(word) # refactorizar
    return _.strip()

In [297]:
# refactorizar
def generate_data_ner(text):
    list_label = []
    list_value = []
    index = 0
    data = text.split(' ')
    for item in data:
        clean_item = clean_word(item)
        if clean_item == "":
            pass
        elif validate_label.get(clean_item):
            list_label.append(validate_label.get(clean_item))
            list_value.append(clean_item)
        elif clean_item.isnumeric():
            list_label.append(validate_label.get('num'))
            list_value.append(clean_item)
        elif len(item) <= 3 and item != 'ley':
            list_label.append(validate_label_mix.get('sw'))
            list_value.append(clean_item)
        else:
            list_label.append(validate_label.get(False))
            list_value.append(clean_item)
        index = index + 1
        
    NER = dict(
        zip(
            list_value, list_label
        )
    )
        
    print("=========== VALUES ===============")
    print(list_value)   
    print("=========== LABELS ===============")
    print(list_label) 
    print("=========== NER - DICT ===============")
    print(NER)
    return NER

In [298]:
generate_data_ner(s)

=========== VALUES ===============
['15', 'el', 'articulo', '86', 'de', 'la', 'c', 'p', 'reza', 'lo', 'siguiente', 'toda', 'persona', 'tendra', 'accion', 'de', 'articulo', '1223', '2323', 'tutela', 'para', 'reclamar', 'ante', 'los', 'jueces', 'del', 'folio', 'del', 'decreto']
=========== LABELS ===============
['NUM', 'SW', 'B-ART', 'NUM', 'SW', 'SW', 'SW', 'SW', 'MIS', 'SW', 'MIS', 'MIS', 'MIS', 'MIS', 'MIS', 'SW', 'B-ART', 'NUM', 'NUM', 'MIS', 'MIS', 'MIS', 'MIS', 'SW', 'MIS', 'SW', 'FOL', 'SW', 'DEC']
=========== NER - DICT ===============
{'15': 'NUM', 'el': 'SW', 'articulo': 'B-ART', '86': 'NUM', 'de': 'SW', 'la': 'SW', 'c': 'SW', 'p': 'SW', 'reza': 'MIS', 'lo': 'SW', 'siguiente': 'MIS', 'toda': 'MIS', 'persona': 'MIS', 'tendra': 'MIS', 'accion': 'MIS', '1223': 'NUM', '2323': 'NUM', 'tutela': 'MIS', 'para': 'MIS', 'reclamar': 'MIS', 'ante': 'MIS', 'los': 'SW', 'jueces': 'MIS', 'del': 'SW', 'folio': 'FOL', 'decreto': 'DEC'}


{'15': 'NUM',
 'el': 'SW',
 'articulo': 'B-ART',
 '86': 'NUM',
 'de': 'SW',
 'la': 'SW',
 'c': 'SW',
 'p': 'SW',
 'reza': 'MIS',
 'lo': 'SW',
 'siguiente': 'MIS',
 'toda': 'MIS',
 'persona': 'MIS',
 'tendra': 'MIS',
 'accion': 'MIS',
 '1223': 'NUM',
 '2323': 'NUM',
 'tutela': 'MIS',
 'para': 'MIS',
 'reclamar': 'MIS',
 'ante': 'MIS',
 'los': 'SW',
 'jueces': 'MIS',
 'del': 'SW',
 'folio': 'FOL',
 'decreto': 'DEC'}

## ============> Dinamyc 

In [266]:
for index, row in decision_new_data_articulo[1:2].iterrows():
    generate_data_ner(row['articulos'])


['articulo', '319', 'del', 'codigo', 'de', 'procedimiento', 'civil']
['ART', 'NUM', 'SW', 'MIS', 'SW', 'MIS', 'MIS']


In [249]:
decision_new_data_articulo[1:2]['articulos'][1].split(' ')

['articulo', '', '319', 'del', 'codigo', 'de', 'procedimiento', 'civil']

In [240]:
decision_new_data_articulo[1:2]['articulos']

1    articulo  319 del codigo de procedimiento civil
Name: articulos, dtype: object